In [1]:
import torch
import torch.nn as nn

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.bn = nn.BatchNorm2d(growth_rate)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False)
        
    def forward(self, x):
        out = self.relu(self.bn(self.conv(x)))
        return torch.cat([x, out], 1)

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList([
            DenseLayer(in_channels + i * growth_rate, growth_rate) 
            for i in range(num_layers)
        ])
        
    def forward(self, x):
        for layer in self.layers:
            print(layer)
            x = layer(x)
        return x


In [4]:
x = torch.randn(1, 3, 224, 224)
x.shape

torch.Size([1, 3, 224, 224])

In [5]:
model = DenseBlock(4,3,12)
out = model(x)
out.shape

DenseLayer(
  (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(15, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(27, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(39, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)


torch.Size([1, 51, 224, 224])

In [6]:
import torch.nn.functional as F
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.pool(self.relu(self.bn(self.conv(x))))
        return x


In [7]:
model = TransitionLayer(3, 6)
out =model(x)
out.shape

torch.Size([1, 6, 112, 112])

In [8]:
class DenseNet(nn.Module):
    def __init__(self, growth_rate = 32, block_config=[6, 12, 24, 16], num_init_features=64, bn_size=4, num_classes=1000 ):
        super(DenseNet, self).__init__()

        # Initial Convolution
        self.conv1 = nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(num_init_features)
        self.relu = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Dense Blocks and Trasition Layers
        num_features = num_init_features
        self.layers = nn.ModuleList()
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(num_layers, num_features, growth_rate)
            self.layers.append(block)
            num_features += num_layers * growth_rate

            if i != len(block_config) - 1:
                self.layers.append(TransitionLayer(num_features, num_features//2))
                num_features = num_features // 2
            
        # Final Batch Norm Layer
        self.bn2 = nn.BatchNorm2d(num_features)
        
        # Fully Connected Layer for Classification
        self.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):

        x = self.pool1(self.relu(self.bn1(self.conv1(x))))

        # Pass through the dense blocks and transition layers
        for layer in self.layers:
            x = layer(x)

        # Final batch normalization and global average pooling
        x = self.relu(self.bn2(x))
        print(x.shape)
        x = nn.AdaptiveAvgPool2d(1)(x)  # Global Average Pooling

        # Flatten and pass through the fully connected layer
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
        

In [9]:
block = {'DenseNet121':[6,12,24,16],
         'DenseNet169':[6,12,32,32],
         'DenseNet201':[6,12,48,32],
         'DenseNet264':[6,12,64,48]
        }
         

In [10]:
model = DenseNet(growth_rate=32, block_config=[6, 12, 24, 16], num_init_features=64, num_classes=1000)
print(model)

DenseNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): ModuleList(
    (0): DenseBlock(
      (layers): ModuleList(
        (0): DenseLayer(
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): DenseLayer(
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (2): DenseLayer(
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, tr

In [12]:
out = model(x)
out

DenseLayer(
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)
DenseLayer(
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace

tensor([[ 9.7668e-03, -3.7860e-01, -1.6391e-01,  1.9757e-01,  4.5972e-01,
          1.1267e-01,  1.8375e-01, -4.2774e-01, -4.0939e-03, -2.3928e-01,
          3.3555e-03, -4.2352e-01,  5.7110e-02,  8.7372e-02, -1.9397e-01,
         -3.3644e-01,  2.7144e-01,  1.4336e-01,  7.1669e-02,  2.6144e-01,
         -3.4123e-01, -4.0382e-02, -1.6076e-01, -4.7710e-02, -1.3829e-02,
         -4.6253e-02, -2.3623e-01, -2.8203e-01,  6.8277e-02,  2.2514e-01,
         -1.6119e-01, -2.5847e-01, -1.5928e-01, -3.7584e-02, -3.1216e-02,
         -7.7512e-03,  1.8265e-02,  3.2286e-01, -2.6980e-01,  3.7404e-01,
         -7.1474e-02,  1.4882e-01, -1.0024e-01,  2.3836e-01,  2.5924e-01,
         -1.1057e-01,  2.0306e-01, -2.1849e-02, -1.2483e-01,  3.7845e-02,
         -1.3788e-01, -2.3662e-01,  6.0554e-02, -4.8630e-01,  1.3196e-01,
          3.6801e-02,  1.6690e-01, -1.7089e-02,  6.0908e-02,  1.4935e-01,
          5.2179e-02, -1.1079e-01, -1.8786e-01, -2.0665e-01,  1.0402e-01,
          9.6173e-02, -2.3212e-01,  3.

In [34]:
out = nn.AdaptiveAvgPool2d(1)(x)
out.shape

torch.Size([1, 3, 1, 1])

In [33]:
out.view(out.size(0) ,-1)

tensor([[-0.0015,  0.0068,  0.0008]])

In [32]:
x.size(0)

1

In [9]:
for i, j in enumerate(block['DenseNet169']):
    print(i,j)

0 6
1 12
2 32
3 32


In [10]:
block_config=(6, 12, 24, 16)

In [12]:
len(block_config) -1

3

In [27]:
x.shape

torch.Size([1, 3, 224, 224])

In [39]:
[ i for i in range(4)]


[0, 1, 2, 3]

In [30]:

x.shape

torch.Size([1, 3, 224, 224])

In [36]:
torch.cat([x,x],-1).shape

torch.Size([1, 3, 224, 448])

In [9]:
torch.manual_seed(0)
y = torch.randint(1,16, (4,4)).reshape(1,1,4,4)
y


tensor([[[[15, 10,  9,  1],
          [14, 10,  8, 14],
          [ 8,  4, 12, 12],
          [ 7, 10, 14, 12]]]])

In [20]:
y1 = torch.rand(16).reshape(1,1,4,4)
y1

tensor([[[[0.6977, 0.8000, 0.1610, 0.2823],
          [0.6816, 0.9152, 0.3971, 0.8742],
          [0.4194, 0.5529, 0.9527, 0.0362],
          [0.1852, 0.3734, 0.3051, 0.9320]]]])

In [21]:
class Pool(nn.Module):
    def __init__(self):
        super(Pool, self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(kernel_size=2, stride=2)
        self.adaptive = nn.AdaptiveAvgPool2d(

    def forward(self, x, pool ):
        if pool == "Max":
             return self.pool(x)
        elif pool == "Avg":
             return self.avg(x)
        
p = Pool()
p(y1, "Avg")

tensor([[[[0.7736, 0.4286],
          [0.3827, 0.5565]]]])

In [15]:
y = torch.rand(1, 1, 4, 4)
y.shape

torch.Size([1, 1, 4, 4])

In [17]:
y

tensor([[[[0.2387, 0.2952, 0.3596, 0.4927],
          [0.2543, 0.0102, 0.7773, 0.5487],
          [0.7067, 0.2236, 0.2989, 0.0781],
          [0.0255, 0.8643, 0.2114, 0.2218]]]])

In [16]:
pool = nn.MaxPool2d(kernel_size=2, stride=2)
pool(y)

tensor([[[[0.2952, 0.7773],
          [0.8643, 0.2989]]]])

In [18]:
pool = nn.AvgPool2d(kernel_size=2, stride=2)
pool(y)

tensor([[[[0.1996, 0.5446],
          [0.4550, 0.2026]]]])

In [19]:
pool = nn.AdaptiveAvgPool2d((2,2))
pool(y)

tensor([[[[0.1996, 0.5446],
          [0.4550, 0.2026]]]])

In [20]:
pool = nn.AdaptiveAvgPool2d((1,1))
pool(y)

tensor([[[[0.3504]]]])

In [24]:
x = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
result = torch.sum(x, dim=(1, 2))  # Sum over dimensions 1 and 2
print("Tensor:", x)
print("Sum over multiple dimensions:", result)

Tensor: tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
Sum over multiple dimensions: tensor([10, 26])


In [22]:
x.shape

torch.Size([2, 2, 2])

In [29]:
y.view(-1).sum()/len(y.view(-1))

tensor(0.3504)

In [31]:
x

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

In [30]:
x.view(-1)

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [32]:
x.flatten()

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [33]:
x.reshape(-1)

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [36]:
x.ravel()

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [37]:
x.is_contiguous()

True

In [11]:
17//2

8

In [12]:
19.5//2


9.0